# **Chapter 6: Precognition (Thinking Step by Step)**

---
**Lesson:**

When you’re faced with a challenging question, taking a moment to think it through often leads to a better answer. Llama3 benefits from the same approach. For intricate tasks, guiding Llama3 to consider each step methodically can greatly enhance its accuracy. By structuring the problem-solving process, Llama3 can deliver more precise and thoughtful responses.

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = 'meta.llama3-8b-instruct-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 512,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

**Example 6.1 - Movie Review**

In the movie review prompt below, it's clear to a human reader that the second sentence belies the first. But Llama3 takes the word "unrelated" too literally.

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since the year 1900.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 6.2 - Better movie review**

To improve Llama3's response, let's allow Llama3 to think things out first before answering. We do that by literally spelling out the steps that Llama3 should take in order to process and think through the prompt. 
Now, Llama3's response is much better and more nuanced.

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Is this review sentiment positive or negative? First write the best arguments for each side in <positive-arg> and <negative-arg> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 6.3 - Blockbusters**

Letting Llama3 think can shift Llama3's answer from incorrect to correct. It's that simple in many cases where Llama3 makes mistakes!
Let's go through an example where Llama3's answer is incorrect to see how asking Llama3 to think can fix that.	

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Are both directors of Jaws and Casino Royale citizens of the same country?
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 6.4 - Blockbusters 2**

Let's fix this by asking Llama3 to think step by step. Instead of spelling out each step for Llama3, you can also simply use those exact words and ask Llama3 to think step by step first.

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Are both directors of Jaws and Casino Royale citizens of the same country? Think step by step and write it out.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 6.5 - Simon says**

It's important to note that to some degree, "let's think step by step" has been pre-built into Llama3's training. You will sometimes see Llama3 thinking step by step even when you don't ask it to, particularly if you asked it a logic or math problem. However, even then, such techniques are not failproof, and Llama3 thinks step by step and still gets the wrong answer. To help Llama3 here, we can tell Llama3 exactly what process it should undertake to think through the problem.

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Use the following clues to answer the following multiple-choice question, using the following procedure:
(1) First, go through the clues one by one and consider whether the clue is potentially relevant
(2) Second, combine the relevant clues to reason out the answer to the question
(3) Third, map the answer to one of the multiple choice answers: either (a), (b), or (c)
 
Clues:
1. Miss Scarlett was the only person in the lounge.
2. The person with the pipe was in the kitchen.
3. Colonel Mustard was the only person in the observatory.
4. Professor Plum was not in the library nor the billiard room.
5. The person with the candlestick was in the observatory.
 
Question: Was Colonel Mustard in the observatory with the candlestick?
(a) Yes; Colonel Mustard was in the observatory with the candlestick
(b) No; Colonel Mustard was not in the observatory with the candlestick
(c) Unknown; there is not enough information to determine whether Colonel Mustard was in the observatory with the candlestick.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 6.1 - Classifying Emails**

In this exercise, we'll be instructing Llama3 to sort emails into the following categories:
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)

For the first part of the exercise, change the prompt in the cell below to make Llama3 output the correct classification. Your answer needs to include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please classify this email: 

    <email>
    {EMAIL} 
    </email>

    Here are the 4 categories it can be classified into, you must choose one:
    (A) Pre-sale question
    (B) Broken or defective item
    (C) Billing question 
    (D) Other (please explain)
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(EMAIL="Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.") #Should be classified as (B)

response = invoke_model_and_get_response(prompt)
print(response)

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please classify this email: 

    <email>
    {EMAIL} 
    </email>

    Here are the 4 categories it can be classified into, you must choose one:
    (A) Pre-sale question
    (B) Broken or defective item
    (C) Billing question 
    (D) Other (please explain)
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(EMAIL="I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WHAT IS GOING ON???") #Should be classified as (C)

response = invoke_model_and_get_response(prompt)
print(response)

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please classify this email: 

    <email>
    {EMAIL} 
    </email>

    Here are the 4 categories it can be classified into, you must choose one:
    (A) Pre-sale question
    (B) Broken or defective item
    (C) Billing question 
    (D) Other (please explain)
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(EMAIL="How did I get here I am not good with computer. Halp.") #Should be classified as (D)

response = invoke_model_and_get_response(prompt)
print(response)

# Chapter 6 - END.